# Bibliotecas

In [10]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pickle
import os
import asyncio
import nest_asyncio
import random
import pandas as pd
from datetime import datetime

#para manipular google sheets
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

# Variáveis Estáticas e autenticação JSON
### Algumas funções usarão implicitamente estas variáveis, então é preciso defini-las antes de rodar o código
O github não permite o envio de arquivos sensíveis como JSON com dados de autenticação, entao armazene localmente o arquivo

In [ ]:
#Google Sheets API
#Google Drive API
ID_DA_PLANILHA_GOOGLE_SHEETS = '1TneLohY9QvCDMCJAS0fCdNXsqxEnVw-YsRl3DpzVDdY' #fica entre /d/ e /edit na URL.

#ARQUIVO DE PERTENCIMENTO DA FEADEV
LOCAL_CREDENCIAIS_JSON_DE_ACESSO = 'E:/Users/Tiago/Feadev/Projetos/Mae DEV/credenciais.json' #Coloque o endereço do arquivo no seu PC


# Escopos de acesso
ESCOPOS = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]



# Criação de um template do excel para manipulação de dados
O template criado é um exemplo da estrutura de dados para armazenar membros, trilhas, cursos e suas respectivas associações

In [13]:
#tome muito cuidado para não sobrescresver o arquivo populado
NOME_TEMPLATE = 'TEMPLATE.xlsx'

membros_feadev = pd.DataFrame({
        'id_membro': [0, 1, 2],
        'nome': ['admin', 'Tiago Toledo', 'Rogério Ceni'],
        'email': ['email-falso-admin-dev@gmail.com', 'ttduarte@usp.br', 'rogerio-ceni@gmail.com'],
        'conta_github': ['admin-feadev-github', 'Tiago745', 'Rogerio-Ceni-Github'],
        'conta_datacamp': ['', '', ''], #provavelmente será uma url para cursos finalizados, ainda precisa ser revisado
        'xp_datacamp': ['', '', ''],
        'ativo': [1, 1, 0]
})

trilhas = pd.DataFrame({
        'id_trilha': [0, 1, 2], #chave/id/pk
        'nome_trilha': ['Associate Data Scientist in Python', 'Associate Data Engineer in SQL',  'Capacitação 2025 - Básico'],
        'url': ["https://app.datacamp.com/learn/career-tracks/associate-data-scientist-in-python","https://app.datacamp.com/learn/career-tracks/associate-data-engineer-in-sql", ''],
        'tipo_trilha': [0,0,1] # 0-> trilha do datacamp || 1-> trilha personalizada (default deve ser 0, 1 somente quando for personalizado ou criado in-app)
        })

cursos = pd.DataFrame({
    'id_curso': [0, 1, 2],
    'nome_curso': ['Introduction to Python', 'Intermediate Python', 'SQL'],
    'duracao' : [4, 4, 4],
    'url' : ['', '', '']
})

eventos = pd.DataFrame({
    'id_evento': [0],
    'nome_evento': ['QuantiConnect 2025 - Dia 1'],
    'tipo_evento_id': ['2'], #PK/CHAVE ESTRANGEIRA DE Tipos_de_Eventos
    'tipo_evento_nome': ['Feira'], #PK/CHAVE ESTRANGEIRA DE Tipos_de_Eventos
    'descricao': ['Primeiro dia do QuantiConnect 2025, evento organizado pela Feadev com foco em inovação quantitativa, networking e palestras.'],
    'data_inicio' : [pd.to_datetime('2025-05-10 10:00')],
    'data_fim' : [pd.to_datetime('2025-05-10 17:00')],
})

tipos_de_eventos = pd.DataFrame({
    'id_tipo_de_evento': [0, 1, 2],
    'nome_tipo_de_evento': ['Reunião', 'Apresentação', 'Feira'],
})



#DataFrames Associativos
trilhas_tem_cursos = pd.DataFrame({
    'id_trilha' : [0, 0, 1, 2, 2, 2],
    'id_curso' : [0, 1, 2, 0, 1, 2],
    'ordem_curso' : [0, 1, 0, 1, 2, 0], #ordem em que os cursos devem ser assistidos dentro de cada trilha
    'data_final_para_assistir': ['', '', '','20/06/2023', '20/06/2023', '23/06/2023'], #cursos de trilhas do datacamp nao tem data-final, apenas trilhas personalizadas
    'obrigatoriedade_curso': [1, 1, 1, 1, 1, 1] #o curso precisa ser obrigatoriamente assistido ou é opcional, 1->Obrigatório 0->Não obrigatório/opcional
})

membro_feadev_faz_trilhas = pd.DataFrame({
    'id_membro' : [1],
    'id_trilha' : [2],
    'data_inicio' : ['15/06/2025'],
    'data_fim' : ['20/06/2025'],
    'finalizado' : [True]
})

membro_feadev_faz_cursos = pd.DataFrame({
    'id_membro' : [1, 1, 1],
    'id_curso' : [0,1,2],
    'data_inicio' : ['15/06/2025', '17/06/2025', '20/06/2025'],
    'data_fim' : ['15/06/2025', '21/06/2025', '21/06/2025'],
    'finalizado' : [True, True, True]
})

membro_feadev_participa_eventos = pd.DataFrame({
    'id_membro' : [0],
    'id_evento' : [0],
    'presença' : [1], #0 = Ausente, 1 = Presente
})

#Uniao de trilhas com cursos
resultado = trilhas.merge(trilhas_tem_cursos, on='id_trilha') \
                   .merge(cursos, on='id_curso') \
                   #[['id_trilha', 'nome_trilha', 'id_curso', 'nome_curso']]  # remove colunas extras

resultado_ordenado = resultado.sort_values(by=['id_trilha', 'ordem_curso']) #ordena por trilha e depois por ordem que deve assistir cada curso, facilita a leitura

print(resultado_ordenado[['id_trilha', 'nome_trilha', 'id_curso', 'nome_curso', 'ordem_curso']])

#limpando as linhas de lorem ipsum em trilhas e cursos
trilhas = trilhas[0:0]
cursos = cursos[0:0]

# Caminho do arquivo Excel que será salvo
arquivo_excel = 'dados_trilhas.xlsx'

# Usando o ExcelWriter para salvar cada DataFrame em uma aba diferente
with pd.ExcelWriter(NOME_TEMPLATE, engine='openpyxl') as writer:
    trilhas.to_excel(writer, sheet_name='trilhas', index=False)
    cursos.to_excel(writer, sheet_name='cursos', index=False)
    eventos.to_excel(writer, sheet_name='eventos', index=False)
    tipos_de_eventos.to_excel(writer, sheet_name='tipos_de_eventos', index=False)
    trilhas_tem_cursos.to_excel(writer, sheet_name='trilhas_tem_cursos', index=False)
    membros_feadev.to_excel(writer, sheet_name='membros_feadev', index=False)
    membro_feadev_faz_trilhas.to_excel(writer, sheet_name='membro_feadev_faz_trilhas', index=False)
    membro_feadev_faz_cursos.to_excel(writer, sheet_name='membro_feadev_faz_cursos', index=False)
    membro_feadev_participa_eventos.to_excel(writer, sheet_name='membro_feadev_participa_eventos', index=False)

print("✅ Arquivo Excel salvo com sucesso!")


   id_trilha                         nome_trilha  id_curso  \
0          0  Associate Data Scientist in Python         0   
1          0  Associate Data Scientist in Python         1   
2          1      Associate Data Engineer in SQL         2   
5          2           Capacitação 2025 - Básico         2   
3          2           Capacitação 2025 - Básico         0   
4          2           Capacitação 2025 - Básico         1   

               nome_curso  ordem_curso  
0  Introduction to Python            0  
1     Intermediate Python            1  
2                     SQL            0  
5                     SQL            0  
3  Introduction to Python            1  
4     Intermediate Python            2  
✅ Arquivo Excel salvo com sucesso!


# Funções para manipular os dados
- Atualizar trilhas e cursos do datacamp, associação entre cursos e trilhas
- Registro de presença em eventos da Feadev
- Registra interações entre membros da Feadev e cursos ou trilhas
- Funções de listagem

In [58]:
#nome do arquivo em que está contido o banco de dados do maedev
NOME_ARQUIVO = 'arquivo.xlsx'
PASTA_TRILHAS = '../trilhas'

#Recupera o nome de cada arquivo dentro da subpasta trilhas_atualizadas (ISSO É SEMPRE ARQUIVO LOCAL, NUNCA NA NUVEM)
LISTA_DE_TRILHAS_PARA_LER = [f for f in os.listdir(PASTA_TRILHAS) if os.path.isfile(os.path.join(PASTA_TRILHAS, f))]

#Gera um cliente autenticado (para facilitar a manipulação de sessão)
def abrir_cliente_JSON():
    # Autenticação usando o JSON baixado
    credenciais = ServiceAccountCredentials.from_json_keyfile_name(LOCAL_CREDENCIAIS_JSON_DE_ACESSO, ESCOPOS) #Credenciais do JSON

    cliente = gspread.authorize(credenciais)
    return cliente

def atualizar_trilhas_com_ids(trilhas_atualizadas_datacamp, maedev_df, col_nome_trilha_atualizada="Trilha", col_df_maedev=['id_trilha', 'nome_trilha', 'url', 'tipo_trilha']):
    """
    Adiciona novas trilhas únicas de `trilhas_atualizados_datacamp` em `maedev_df`, com id incremental.
    
    Parâmetros:
        trilhas_atualizados_datacamp (pd.DataFrame): DataFrame com os cursos e suas trilhas.
        maedev_df (pd.DataFrame): DataFrame com trilhas já cadastradas.
        col_nome_trilha_atualizada (str): Nome da trilha no trilhass_atualizadas_datacamp que contém o nome das trilhas.
        col_df_maedev (list): Lista com os nomes das colunas na ordem: [id, nome, url, tipo].

    Retorna:
        pd.DataFrame: O DataFrame `maedev_df` atualizado com novas trilhas e IDs únicos.
    """
    url_padrao_trilha = 'https://app.datacamp.com/learn/career-tracks/'
    
    # Padroniza os nomes das trilhas
    trilhas_atualizadas_datacamp[col_nome_trilha_atualizada] = trilhas_atualizadas_datacamp[col_nome_trilha_atualizada].astype(str).str.strip()
    maedev_df[col_df_maedev[1]] = maedev_df[col_df_maedev[1]].astype(str).str.strip()

    # ID inicial baseado no maior ID atual
    ultimo_id = maedev_df[col_df_maedev[0]].max() if not maedev_df.empty else -1
    novas_linhas = []



     # Itera cada coluna unica em cursos atualizados(apenas combinacoes unicas), depois acessa o valor nas colunas nome_curso e Duração
    for _, row in trilhas_atualizadas_datacamp[[col_nome_trilha_atualizada, 'tipo_trilha']].drop_duplicates().iterrows():
        trilha = row[col_nome_trilha_atualizada]
        tipo_da_trilha = row['tipo_trilha'] #0-> trilha oficial do datacamp, 1-> trilha personalizada

        #Se essa trilha não está inclusa no maedev, adicona ela a lista de novos trilhas
        if trilha not in maedev_df[col_df_maedev[1]].values:
            ultimo_id += 1
            novas_linhas.append({
                col_df_maedev[0]: ultimo_id, #id
                col_df_maedev[1]: trilha, #nome da trilha
                col_df_maedev[2]: url_padrao_trilha + "-".join(trilha.lower().split()), #url da trilha
                col_df_maedev[3]: tipo_da_trilha
            })
            print(f"✅ Novo curso adicionado: {trilha} (id = {ultimo_id})")

    # Adiciona as novas trilhas, se houver
    if novas_linhas:
        df_novos = pd.DataFrame(novas_linhas)
        maedev_df = pd.concat([maedev_df, df_novos], ignore_index=True)
        print("🎯 Atualização concluída com sucesso.")
    else:
        print("ℹ️ Nenhuma nova trilha encontrada.")

    return maedev_df

def atualizar_cursos_com_ids(cursos_atualizados_datacamp, maedev_df, col_nome_curso_atualizada="Curso", col_df_maedev=['id_curso', 'nome_curso', 'duracao', 'url']):
    """
    Adiciona novas trilhas únicas de `cursos_atualizados_datacamp` em `maedev_df`, com id incremental.
    
    Parâmetros:
        cursos_atualizados_datacamp (pd.DataFrame): DataFrame com os cursos e suas trilhas.
        maedev_df (pd.DataFrame): DataFrame com trilhas já cadastradas.
        col_nome_curso_atualizada (str): Nome da coluna no cursos_atualizados_datacamp que contém o nome das trilhas.
        col_df_maedev (list): Lista com os nomes das colunas na ordem: [id, nome, url, tipo].

    Retorna:
        pd.DataFrame: O DataFrame `maedev_df` atualizado com novas trilhas e IDs únicos.
    """
    url_padrao_trilha = 'https://app.datacamp.com/learn/courses/'
    
    # Padroniza os nomes dos cursos
    cursos_atualizados_datacamp[col_nome_curso_atualizada] = cursos_atualizados_datacamp[col_nome_curso_atualizada].astype(str).str.strip()
    maedev_df[col_df_maedev[1]] = maedev_df[col_df_maedev[1]].astype(str).str.strip()

    # ID inicial baseado no maior ID atual
    ultimo_id = maedev_df[col_df_maedev[0]].max() if not maedev_df.empty else -1
    novas_linhas = []


    # Itera cada coluna unica em cursos atualizados(apenas combinacoes unicas), depois acessa o valor nas colunas nome_curso e Duração
    for _, row in cursos_atualizados_datacamp[[col_nome_curso_atualizada, 'Duração']].drop_duplicates().iterrows():
        curso = row[col_nome_curso_atualizada]
        duracao = row['Duração']

        #Se esse curso não está incluso no maedev, adicona ele a lista de novos cursos
        if curso not in maedev_df[col_df_maedev[1]].values:
            ultimo_id += 1
            novas_linhas.append({
                col_df_maedev[0]: ultimo_id,
                col_df_maedev[1]: curso,
                col_df_maedev[2]: duracao,
                col_df_maedev[3]: url_padrao_trilha + "-".join(curso.lower().split()),
            })
            print(f"✅ Novo curso adicionado: {curso} (id = {ultimo_id})")

    # Adiciona os novos cursos, se houver
    if novas_linhas:
        df_novos = pd.DataFrame(novas_linhas)
        maedev_df = pd.concat([maedev_df, df_novos], ignore_index=True)
        print("🎯 Atualização concluída com sucesso.")
    else:
        print("ℹ️ Nenhum novo curso encontrado.")

    return maedev_df

#Para associar as trilhas com os cursos/ gerar associações
def gerar_trilhas_tem_cursos(cliente, id_key_google_sheets=ID_DA_PLANILHA_GOOGLE_SHEETS, pasta_trilhas=PASTA_TRILHAS):
    """
    Gera a sheet 'Trilhas_tem_Cursos' na planilha Google especificada pelo ID, associando cada trilha aos cursos correspondentes.

    A função percorre todos os arquivos .xlsx dentro da pasta fornecida, assumindo que cada arquivo representa uma trilha
    (coluna 'Trilha') com os cursos correspondentes (coluna 'Curso'). Com base nesses nomes, ela busca os respectivos 
    `id_trilha` e `id_curso` nas sheets 'Trilhas' e 'Cursos' da planilha Google, criando a associação com campos extras 
    padrão.

    A nova sheet gerada contém as seguintes colunas:
        - id_trilha (int)
        - id_curso (int)
        - data_final_para_assistir (str): sempre vazio ('')
        - obrigatoriedade_curso (int): sempre 1

    Parâmetros:
    ----------
    cliente : gspread.Client
    Cliente autenticado gspread para acessar a planilha Google.

    id_key_google_sheets : str
        ID da planilha Google Sheets.

    pasta_trilhas : str
        Caminho da pasta contendo os arquivos .xlsx com as trilhas e cursos atualizados.



    Retorno:
    -------
    None
        A função salva a nova sheet 'Trilhas_tem_Cursos' diretamente na planilha Google.
    """

    # Abre a planilha Google pelo ID 
    planilha = cliente.open_by_key(id_key_google_sheets) 

    # Lê os dados principais da planilha Google
    aba_trilhas = planilha.worksheet('trilhas')
    aba_cursos = planilha.worksheet('cursos')
    df_trilhas = get_as_dataframe(aba_trilhas).dropna(how='all') #Transforma em DF
    df_cursos = get_as_dataframe(aba_cursos).dropna(how='all') #Transforma em DF

    associacoes = []

    # Itera sobre os arquivos de trilha atualizada (arquivo local)
    for nome_arquivo in os.listdir(pasta_trilhas):
        if not nome_arquivo.endswith(".xlsx"):
            continue

        caminho = os.path.join(pasta_trilhas, nome_arquivo)
        df = pd.read_excel(caminho)

        # Extrai os nomes da trilha e cursos
        nome_trilha = str(df['trilha'].iloc[0]).strip()

        # Recupera o id_trilha
        linha_trilha = df_trilhas[df_trilhas['nome_trilha'].str.strip() == nome_trilha]
        if linha_trilha.empty:
            print(f"❌ trilha não encontrada: {nome_trilha}")
            continue
        id_trilha = int(linha_trilha['id_trilha'].values[0])

        for nome_curso in df['curso'].unique():
            nome_curso = str(nome_curso).strip()
            linha_curso = df_cursos[df_cursos['nome_curso'].str.strip() == nome_curso]

            if linha_curso.empty:
                print(f"⚠️ curso não encontrado: {nome_curso}")
                continue

            id_curso = int(linha_curso['id_curso'].values[0])

            associacoes.append({
                'id_trilha': id_trilha,
                'id_curso': id_curso,
                'data_final_para_assistir': '',
                'obrigatoriedade_curso': 1
            })

    # Cria dataframe da associação
    df_associacoes = pd.DataFrame(associacoes)


    # Atualiza ou cria a aba 'Trilhas_tem_Cursos' na planilha Google
    try:
        aba_assoc = planilha.worksheet('trilhas_tem_cursos')
        aba_assoc.clear()  #Limpa a aba caso ela exista
    except Exception:
        # Cria nova aba se não existir
        aba_assoc = planilha.add_worksheet(title='trilhas_tem_cursos', rows=str(len(df_associacoes)+10), cols='10')


    # Escreve o dataframe na aba
    from gspread_dataframe import set_with_dataframe
    set_with_dataframe(aba_assoc, df_associacoes, include_index=False)

#Para marcar que um usuário x esteve presente ou ausente no evento y, também serve para atualizar o registro
#Essa função cria se necessário e atualiza o registro caso já exista
def registrar_participacao_evento(id_key_google_sheets, cliente, id_membro, id_evento, presenca=1):
    """
    Registra a participação de um membro da Feadev em um evento na planilha Google.

    A função acessa a aba 'membro_feadev_participa_eventos', e atualiza ou adiciona a participação,
    e salva de volta na mesma aba.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    id_membro : int
        ID do membro participante.

    id_evento : int
        ID do evento.

    presenca : int, opcional (padrão=1)
        1 para presente, 0 para ausente.

    Retorno:
    --------
    None
    """
    # Abre a planilha
    planilha = cliente.open_by_key(id_key_google_sheets)

    # Tenta abrir a aba, ou cria uma nova se não existir
    try:
        aba = planilha.worksheet('membro_feadev_participa_eventos')
        df_participacoes = get_as_dataframe(aba).dropna(how='all')
    except:
        aba = planilha.add_worksheet(title='membro_feadev_participa_eventos', rows='100', cols='5')
        df_participacoes = pd.DataFrame(columns=['id_membro', 'id_evento', 'presença'])

    # Garante os tipos certos
    df_participacoes['id_membro'] = pd.to_numeric(df_participacoes['id_membro'], errors='coerce').fillna(-1).astype(int)
    df_participacoes['id_evento'] = pd.to_numeric(df_participacoes['id_evento'], errors='coerce').fillna(-1).astype(int)

    # Verifica se já existe
    existe = (
        (df_participacoes['id_membro'] == id_membro) &
        (df_participacoes['id_evento'] == id_evento)
    )

    if existe.any():
        df_participacoes.loc[existe, 'presença'] = presenca
    else:
        nova_linha = pd.DataFrame([{
            'id_membro': id_membro,
            'id_evento': id_evento,
            'presença': presenca
        }])
        df_participacoes = pd.concat([df_participacoes, nova_linha], ignore_index=True)

    # Salva de volta na aba (substitui tudo)
    aba.clear()
    set_with_dataframe(aba, df_participacoes, include_index=False)

#Essa função cria se necessário e atualiza o registro caso já exista
def registrar_membro_faz_curso(id_key_google_sheets, cliente, id_membro, id_curso, data_inicio, data_fim, finalizado=0):
    """
    Registra que um membro da Feadev iniciou ou finalizou um curso.

    A função acessa (ou cria) a aba 'membro_feadev_faz_cursos', atualiza ou adiciona o registro,
    e salva de volta na mesma aba.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    id_membro : int
        ID do membro.

    id_curso : int
        ID do curso.

    data_inicio : str ou datetime
        Data de início do curso.

    data_fim : str ou datetime
        Data de término do curso.

    finalizado : int, opcional (default=0)
        1 para curso finalizado, 0 para não finalizado.

    Retorno:
    --------
    None
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    # Tenta abrir ou criar a aba
    try:
        aba = planilha.worksheet('membro_feadev_faz_cursos')
        df = get_as_dataframe(aba).dropna(how='all')
    except:
        aba = planilha.add_worksheet(title='membro_feadev_faz_cursos', rows='100', cols='6')
        df = pd.DataFrame(columns=[
            'id_membro', 'id_curso', 'data_inicio', 'data_fim', 'finalizado'
        ])

    # Garante tipos numéricos
    df['id_membro'] = pd.to_numeric(df['id_membro'], errors='coerce').fillna(-1).astype(int)
    df['id_curso'] = pd.to_numeric(df['id_curso'], errors='coerce').fillna(-1).astype(int)

    # Converte as datas para datetime (caso venham como string)
    if isinstance(data_inicio, str):
        data_inicio = datetime.fromisoformat(data_inicio)
    if isinstance(data_fim, str):
        data_fim = datetime.fromisoformat(data_fim)

    # Verifica se já existe esse registro
    existe = (
        (df['id_membro'] == id_membro) &
        (df['id_curso'] == id_curso)
    )

    if existe.any():
        # Atualiza o registro existente
        df.loc[existe, 'data_inicio'] = data_inicio
        df.loc[existe, 'data_fim'] = data_fim
        df.loc[existe, 'finalizado'] = finalizado
    else:
        nova_linha = pd.DataFrame([{
            'id_membro': id_membro,
            'id_curso': id_curso,
            'data_inicio': data_inicio,
            'data_fim': data_fim,
            'finalizado': finalizado
        }])
        df = pd.concat([df, nova_linha], ignore_index=True)

    # Salva de volta
    aba.clear()
    set_with_dataframe(aba, df, include_index=False)

#Essa função cria se necessário e atualiza o registro caso já exista
def registrar_membro_faz_trilha(id_key_google_sheets, cliente, id_membro, id_trilha, data_inicio, data_fim, finalizado=0):
    """
    Registra que um membro da Feadev iniciou ou finalizou uma trilha.

    A função acessa (ou cria) a aba 'membro_feadev_faz_trilhas', atualiza ou adiciona o registro,
    e salva de volta na mesma aba.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    id_membro : int
        ID do membro.

    id_trilha : int
        ID da trilha.

    data_inicio : str ou datetime
        Data de início da trilha.

    data_fim : str ou datetime
        Data de término da trilha.

    finalizado : int, opcional (default=0)
        1 para trilha finalizada, 0 para não finalizada.

    Retorno:
    --------
    None
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    # Tenta abrir ou criar a aba
    try:
        aba = planilha.worksheet('membro_feadev_faz_trilhas')
        df = get_as_dataframe(aba).dropna(how='all')
    except:
        aba = planilha.add_worksheet(title='membro_feadev_faz_trilhas', rows='100', cols='6')
        df = pd.DataFrame(columns=[
            'id_membro', 'id_trilha', 'data_inicio', 'data_fim', 'finalizado'
        ])

    # Garante tipos numéricos
    df['id_membro'] = pd.to_numeric(df['id_membro'], errors='coerce').fillna(-1).astype(int)
    df['id_trilha'] = pd.to_numeric(df['id_trilha'], errors='coerce').fillna(-1).astype(int)

    # Converte as datas para datetime (caso venham como string)
    if isinstance(data_inicio, str):
        data_inicio = datetime.fromisoformat(data_inicio)
    if isinstance(data_fim, str):
        data_fim = datetime.fromisoformat(data_fim)

    # Verifica se já existe esse registro
    existe = (
        (df['id_membro'] == id_membro) &
        (df['id_trilha'] == id_trilha)
    )

    if existe.any():
        df.loc[existe, 'data_inicio'] = data_inicio
        df.loc[existe, 'data_fim'] = data_fim
        df.loc[existe, 'finalizado'] = finalizado
    else:
        nova_linha = pd.DataFrame([{
            'id_membro': id_membro,
            'id_trilha': id_trilha,
            'data_inicio': data_inicio,
            'data_fim': data_fim,
            'finalizado': finalizado
        }])
        df = pd.concat([df, nova_linha], ignore_index=True)

    # Salva de volta
    aba.clear()
    set_with_dataframe(aba, df, include_index=False)

def registrar_membro(id_key_google_sheets, cliente, nome, email, conta_github='', conta_datacamp='', xp_datacamp='', ativo=1):
    """
    Registra um novo membro da Feadev na planilha Google na aba 'membros_feadev'.

    Se a aba não existir, ela será criada.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    nome : str
        Nome do membro.

    email : str
        E-mail do membro.

    conta_github : str, opcional
        Nome de usuário no GitHub.

    conta_datacamp : str, opcional
        URL ou identificador da conta no DataCamp.

    xp_datacamp : str, opcional
        Experiência acumulada no DataCamp.

    ativo : int, opcional (default=1)
        Status do membro: 1 para ativo, 0 para inativo.

    Retorno:
    --------
    None
    """
    # Abre a planilha
    planilha = cliente.open_by_key(id_key_google_sheets)

    # Tenta abrir a aba ou cria uma nova se não existir
    try:
        aba = planilha.worksheet('membros_feadev')
        df_membros = get_as_dataframe(aba).dropna(how='all')
    except:
        aba = planilha.add_worksheet(title='membros_feadev', rows='100', cols='7')
        df_membros = pd.DataFrame(columns=[
            'id_membro', 'nome', 'email', 'conta_github',
            'conta_datacamp', 'xp_datacamp', 'ativo'
        ])

    # Garante que os IDs sejam inteiros
    df_membros['id_membro'] = pd.to_numeric(df_membros['id_membro'], errors='coerce').fillna(-1).astype(int)

    # Define o novo ID
    novo_id = df_membros['id_membro'].max() + 1 if not df_membros.empty else 0

    # Cria nova linha
    novo_membro = pd.DataFrame([{
        'id_membro': novo_id,
        'nome': nome,
        'email': email,
        'conta_github': conta_github,
        'conta_datacamp': conta_datacamp,
        'xp_datacamp': xp_datacamp,
        'ativo': ativo
    }])

    # Adiciona e salva
    df_membros = pd.concat([df_membros, novo_membro], ignore_index=True)
    aba.clear()
    set_with_dataframe(aba, df_membros, include_index=False)

def registrar_evento(id_key_google_sheets, cliente, nome_evento, tipo_evento_id='', tipo_evento_nome='', descricao='', data_inicio='', data_fim=''):
    """
    Registra um novo evento da Feadev na aba 'eventos' da planilha Google.

    Se a aba não existir, ela será criada.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    nome_evento : str
        Nome do evento.

    tipo_evento_id : str
        ID do tipo do evento (chave estrangeira).

    tipo_evento_nome : str
        Nome do tipo do evento (chave estrangeira).

    descricao : str
        Descrição do evento.

    data_inicio : datetime.datetime
        Data e hora de início do evento.

    data_fim : datetime.datetime
        Data e hora de fim do evento.

    Retorno:
    --------
    None
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    # Tenta abrir ou cria a aba
    try:
        aba = planilha.worksheet('eventos')
        df_eventos = get_as_dataframe(aba).dropna(how='all')
    except:
        aba = planilha.add_worksheet(title='eventos', rows='100', cols='7')
        df_eventos = pd.DataFrame(columns=[
            'id_evento', 'nome_evento', 'tipo_evento_id', 'tipo_evento_nome',
            'descricao', 'data_inicio', 'data_fim'
        ])

    # Garante que os IDs sejam inteiros
    df_eventos['id_evento'] = pd.to_numeric(df_eventos['id_evento'], errors='coerce').fillna(-1).astype(int)

    # Define o novo ID
    novo_id = df_eventos['id_evento'].max() + 1 if not df_eventos.empty else 0

    # Cria nova linha
    novo_evento = pd.DataFrame([{
        'id_evento': novo_id,
        'nome_evento': nome_evento,
        'tipo_evento_id': tipo_evento_id,
        'tipo_evento_nome': tipo_evento_nome,
        'descricao': descricao,
        'data_inicio': pd.to_datetime(data_inicio),
        'data_fim': pd.to_datetime(data_fim)
    }])

    # Adiciona, limpa e salva de volta
    df_eventos = pd.concat([df_eventos, novo_evento], ignore_index=True)
    aba.clear()
    set_with_dataframe(aba, df_eventos, include_index=False)

def listar_membros(id_key_google_sheets, cliente):
    """
    Retorna um DataFrame com todos os membros registrados na aba 'membros_feadev' da planilha Google.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    Retorno:
    --------
    pd.DataFrame
        DataFrame com os dados dos membros.
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    try:
        aba = planilha.worksheet('membros_feadev')
        df_membros = get_as_dataframe(aba).dropna(how='all')
        return df_membros
    except Exception as e:
        print(f"Erro ao acessar a aba 'membros_feadev': {e}")
        return pd.DataFrame()

def listar_cursos(id_key_google_sheets, cliente):
    """
    Retorna um DataFrame com todos os cursos registrados na aba 'cursos' da planilha Google.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    Retorno:
    --------
    pd.DataFrame
        DataFrame com os dados dos cursos.
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    try:
        aba = planilha.worksheet('cursos')
        df_cursos = get_as_dataframe(aba).dropna(how='all')
        return df_cursos
    except Exception as e:
        print(f"Erro ao acessar a aba 'cursos': {e}")
        return pd.DataFrame()
    
def listar_trilhas(id_key_google_sheets, cliente):
    """
    Retorna um DataFrame com todos as trilhas registrados na aba 'trilhas' da planilha Google.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    Retorno:
    --------
    pd.DataFrame
        DataFrame com os dados dos cursos.
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    try:
        aba = planilha.worksheet('trilhas')
        df_cursos = get_as_dataframe(aba).dropna(how='all')
        return df_cursos
    except Exception as e:
        print(f"Erro ao acessar a aba 'trilhas': {e}")
        return pd.DataFrame()
    
def listar_eventos(id_key_google_sheets, cliente):
    """
    Retorna um DataFrame com todos os eventos registrados na aba 'eventos' da planilha Google.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    Retorno:
    --------
    pd.DataFrame
        DataFrame com os dados dos cursos.
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    try:
        aba = planilha.worksheet('eventos')
        df_cursos = get_as_dataframe(aba).dropna(how='all')
        return df_cursos
    except Exception as e:
        print(f"Erro ao acessar a aba 'eventos': {e}")
        return pd.DataFrame()
    
#Listar/Ler google sheets

#MEMBROS X EVENTOS
def listar_eventos_de_um_usuario(id_key_google_sheets, cliente, id_membro):
    """
    Retorna um DataFrame com os eventos que um membro participou, incluindo o campo presença.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    id_membro : int
        ID do membro para filtrar os eventos.

    Retorno:
    --------
    pd.DataFrame
        DataFrame com os eventos em que o membro participou, contendo também a coluna 'presença'.
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    try:
        # Lê participações do membro
        aba_participacoes = planilha.worksheet('membro_feadev_participa_eventos')
        df_participacoes = get_as_dataframe(aba_participacoes).dropna(how='all')

        # Garantir tipos corretos
        df_participacoes['id_membro'] = pd.to_numeric(df_participacoes['id_membro'], errors='coerce').fillna(-1).astype(int)
        df_participacoes['id_evento'] = pd.to_numeric(df_participacoes['id_evento'], errors='coerce').fillna(-1).astype(int)
        df_participacoes['presença'] = pd.to_numeric(df_participacoes['presença'], errors='coerce').fillna(0).astype(int)

        # Filtra participações do usuário
        participacoes_usuario = df_participacoes[df_participacoes['id_membro'] == id_membro]

        if participacoes_usuario.empty:
            return pd.DataFrame()

        # Lê os eventos
        aba_eventos = planilha.worksheet('eventos')
        df_eventos = get_as_dataframe(aba_eventos).dropna(how='all')
        df_eventos['id_evento'] = pd.to_numeric(df_eventos['id_evento'], errors='coerce').fillna(-1).astype(int)

        # Filtra eventos correspondentes
        eventos_filtrados = df_eventos[df_eventos['id_evento'].isin(participacoes_usuario['id_evento'].unique())]

        # Junta com presença (merge pela coluna id_evento)
        resultado = pd.merge(eventos_filtrados, participacoes_usuario[['id_evento', 'presença']], on='id_evento', how='left')

        return resultado.reset_index(drop=True)

    except Exception as e:
        print(f"Erro ao listar eventos do usuário {id_membro}: {e}")
        return pd.DataFrame()

def listar_participantes_de_um_evento(id_key_google_sheets, cliente, id_evento):
    """
    Retorna um DataFrame com os membros que participaram de um evento, incluindo o campo presença.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    id_evento : int
        ID do evento para filtrar os participantes.

    Retorno:
    --------
    pd.DataFrame
        DataFrame com os membros que participaram do evento, contendo também a coluna 'presença'.
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    try:
        # Lê participações do evento
        aba_participacoes = planilha.worksheet('membro_feadev_participa_eventos')
        df_participacoes = get_as_dataframe(aba_participacoes).dropna(how='all')

        # Garantir tipos corretos
        df_participacoes['id_membro'] = pd.to_numeric(df_participacoes['id_membro'], errors='coerce').fillna(-1).astype(int)
        df_participacoes['id_evento'] = pd.to_numeric(df_participacoes['id_evento'], errors='coerce').fillna(-1).astype(int)
        df_participacoes['presença'] = pd.to_numeric(df_participacoes['presença'], errors='coerce').fillna(0).astype(int)

        # Filtra participações do evento
        participacoes_evento = df_participacoes[df_participacoes['id_evento'] == id_evento]

        if participacoes_evento.empty:
            return pd.DataFrame()

        # Lê os membros
        aba_membros = planilha.worksheet('Membros_feadev')
        df_membros = get_as_dataframe(aba_membros).dropna(how='all')
        df_membros['id_membro'] = pd.to_numeric(df_membros['id_membro'], errors='coerce').fillna(-1).astype(int)

        # Filtra membros correspondentes
        membros_filtrados = df_membros[df_membros['id_membro'].isin(participacoes_evento['id_membro'].unique())]

        # Junta com presença (merge pela coluna id_membro)
        resultado = pd.merge(membros_filtrados, participacoes_evento[['id_membro', 'presença']], on='id_membro', how='left')

        return resultado.reset_index(drop=True)

    except Exception as e:
        print(f"Erro ao listar participantes do evento {id_evento}: {e}")
        return pd.DataFrame()

#MEMBROS X TRILHAS
def listar_trilhas_feitas_por_um_membro(id_key_google_sheets, cliente, id_membro):
    """
    Retorna um DataFrame com as trilhas que um membro está fazendo ou já fez,
    incluindo informações de progresso como data de início, data de fim e se foi finalizado.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    id_membro : int
        ID do membro.

    Retorno:
    --------
    pd.DataFrame
        DataFrame com as trilhas associadas ao membro e suas informações de progresso.
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    try:
        # Lê registros da aba de membros fazendo trilhas
        aba_faz_trilhas = planilha.worksheet('membro_feadev_faz_trilhas')
        df_faz_trilhas = get_as_dataframe(aba_faz_trilhas).dropna(how='all')

        df_faz_trilhas['id_membro'] = pd.to_numeric(df_faz_trilhas['id_membro'], errors='coerce').fillna(-1).astype(int)
        df_faz_trilhas['id_trilha'] = pd.to_numeric(df_faz_trilhas['id_trilha'], errors='coerce').fillna(-1).astype(int)

        # Filtra somente os registros do membro
        trilhas_do_membro = df_faz_trilhas[df_faz_trilhas['id_membro'] == id_membro]

        if trilhas_do_membro.empty:
            return pd.DataFrame()

        # Lê os dados das trilhas
        aba_trilhas = planilha.worksheet('trilhas')
        df_trilhas = get_as_dataframe(aba_trilhas).dropna(how='all')
        df_trilhas['id_trilha'] = pd.to_numeric(df_trilhas['id_trilha'], errors='coerce').fillna(-1).astype(int)

        # Junta os dados da trilha com os dados de progresso do membro
        resultado = trilhas_do_membro.merge(df_trilhas, on='id_trilha', how='left')

        # Organiza as colunas com prioridade para info de progresso
        colunas_base = ['id_trilha', 'nome_trilha', 'url', 'tipo_trilha']
        colunas_progresso = ['data_inicio', 'data_fim', 'finalizado']
        colunas_outros = [c for c in resultado.columns if c not in colunas_base + colunas_progresso + ['id_membro']]

        colunas_final = colunas_base + colunas_progresso + colunas_outros
        return resultado[colunas_final].reset_index(drop=True)

    except Exception as e:
        print(f"Erro ao listar trilhas do membro {id_membro}: {e}")
        return pd.DataFrame()

def listar_membros_de_uma_trilha(id_key_google_sheets, cliente, id_trilha):
    """
    Retorna um DataFrame com os membros que estão fazendo ou fizeram uma trilha específica.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    id_trilha : int
        ID da trilha para filtrar os membros.

    Retorno:
    --------
    pd.DataFrame
        DataFrame com os membros associados à trilha.
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    try:
        # Lê registros da aba de membros fazendo trilhas
        aba_faz_trilhas = planilha.worksheet('membro_feadev_faz_trilhas')
        df_faz_trilhas = get_as_dataframe(aba_faz_trilhas).dropna(how='all')

        df_faz_trilhas['id_membro'] = pd.to_numeric(df_faz_trilhas['id_membro'], errors='coerce').fillna(-1).astype(int)
        df_faz_trilhas['id_trilha'] = pd.to_numeric(df_faz_trilhas['id_trilha'], errors='coerce').fillna(-1).astype(int)

        membros_da_trilha = df_faz_trilhas[df_faz_trilhas['id_trilha'] == id_trilha]

        if membros_da_trilha.empty:
            return pd.DataFrame()

        # Lê os dados dos membros
        aba_membros = planilha.worksheet('membros_feadev')
        df_membros = get_as_dataframe(aba_membros).dropna(how='all')
        df_membros['id_membro'] = pd.to_numeric(df_membros['id_membro'], errors='coerce').fillna(-1).astype(int)

        # Junta os dados dos membros
        membros_completos = df_membros[df_membros['id_membro'].isin(membros_da_trilha['id_membro'].unique())]

        return membros_completos.reset_index(drop=True)

    except Exception as e:
        print(f"Erro ao listar membros da trilha {id_trilha}: {e}")
        return pd.DataFrame()


#MEMBROS X CURSOS
def listar_cursos_feitos_por_um_membro(id_key_google_sheets, cliente, id_membro):
    """
    Retorna um DataFrame com os cursos que um membro está fazendo ou já fez,
    incluindo informações de progresso como data de início, data de fim e se foi finalizado.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    id_membro : int
        ID do membro.

    Retorno:
    --------
    pd.DataFrame
        DataFrame com os cursos associados ao membro e suas informações de progresso.
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    try:
        # Lê registros da aba de membros fazendo cursos
        aba_faz_cursos = planilha.worksheet('membro_feadev_faz_cursos')
        df_faz_cursos = get_as_dataframe(aba_faz_cursos).dropna(how='all')

        df_faz_cursos['id_membro'] = pd.to_numeric(df_faz_cursos['id_membro'], errors='coerce').fillna(-1).astype(int)
        df_faz_cursos['id_curso'] = pd.to_numeric(df_faz_cursos['id_curso'], errors='coerce').fillna(-1).astype(int)

        # Filtra somente os registros do membro
        cursos_do_membro = df_faz_cursos[df_faz_cursos['id_membro'] == id_membro]

        if cursos_do_membro.empty:
            return pd.DataFrame()

        # Lê os dados dos cursos
        aba_cursos = planilha.worksheet('cursos')
        df_cursos = get_as_dataframe(aba_cursos).dropna(how='all')
        df_cursos['id_curso'] = pd.to_numeric(df_cursos['id_curso'], errors='coerce').fillna(-1).astype(int)

        # Junta os dados do curso com os dados de progresso do membro
        resultado = cursos_do_membro.merge(df_cursos, on='id_curso', how='left')

        # Organiza as colunas com prioridade para info de progresso
        colunas_base = ['id_curso', 'nome_curso', 'duracao', 'url']
        colunas_progresso = ['data_inicio', 'data_fim', 'finalizado']
        colunas_outros = [c for c in resultado.columns if c not in colunas_base + colunas_progresso + ['id_membro']]

        colunas_final = colunas_base + colunas_progresso + colunas_outros
        return resultado[colunas_final].reset_index(drop=True)

    except Exception as e:
        print(f"Erro ao listar cursos do membro {id_membro}: {e}")
        return pd.DataFrame()

def listar_membros_de_um_curso(id_key_google_sheets, cliente, id_curso):
    """
    Retorna um DataFrame com os membros que estão fazendo ou fizeram um curso específico.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    id_curso : int
        ID do curso para filtrar os membros.

    Retorno:
    --------
    pd.DataFrame
        DataFrame com os membros associados ao curso.
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    try:
        # Lê registros da aba de membros fazendo cursos
        aba_faz_cursos = planilha.worksheet('membro_feadev_faz_cursos')
        df_faz_cursos = get_as_dataframe(aba_faz_cursos).dropna(how='all')

        df_faz_cursos['id_membro'] = pd.to_numeric(df_faz_cursos['id_membro'], errors='coerce').fillna(-1).astype(int)
        df_faz_cursos['id_curso'] = pd.to_numeric(df_faz_cursos['id_curso'], errors='coerce').fillna(-1).astype(int)

        membros_do_curso = df_faz_cursos[df_faz_cursos['id_curso'] == id_curso]

        if membros_do_curso.empty:
            return pd.DataFrame()

        # Lê os dados dos membros
        aba_membros = planilha.worksheet('membros_feadev')
        df_membros = get_as_dataframe(aba_membros).dropna(how='all')
        df_membros['id_membro'] = pd.to_numeric(df_membros['id_membro'], errors='coerce').fillna(-1).astype(int)

        # Junta os dados dos membros
        membros_completos = df_membros[df_membros['id_membro'].isin(membros_do_curso['id_membro'].unique())]

        return membros_completos.reset_index(drop=True)

    except Exception as e:
        print(f"Erro ao listar membros do curso {id_curso}: {e}")
        return pd.DataFrame()

#CURSOS X TRILHAS
def listar_cursos_de_uma_trilha(id_key_google_sheets, cliente, id_trilha):
    """
    Retorna um DataFrame com todos os cursos associados a uma trilha.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    id_trilha : int
        ID da trilha para filtrar os cursos.

    Retorno:
    --------
    pd.DataFrame
        DataFrame com os cursos associados à trilha.
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    try:
        # Lê associação trilhas -> cursos
        aba_assoc = planilha.worksheet('trilhas_tem_cursos')
        df_assoc = get_as_dataframe(aba_assoc).dropna(how='all')

        # Filtra associações para o id_trilha
        df_assoc['id_trilha'] = pd.to_numeric(df_assoc['id_trilha'], errors='coerce').fillna(-1).astype(int)
        df_assoc['id_curso'] = pd.to_numeric(df_assoc['id_curso'], errors='coerce').fillna(-1).astype(int)
        assoc_filtrada = df_assoc[df_assoc['id_trilha'] == id_trilha]

        if assoc_filtrada.empty:
            print(f"Nenhum curso encontrado para a trilha id {id_trilha}")
            return pd.DataFrame()

        # Lê dados dos cursos
        aba_cursos = planilha.worksheet('cursos')
        df_cursos = get_as_dataframe(aba_cursos).dropna(how='all')
        df_cursos['id_curso'] = pd.to_numeric(df_cursos['id_curso'], errors='coerce').fillna(-1).astype(int)

        # Filtra os cursos que estão associados à trilha
        cursos_trilha = df_cursos[df_cursos['id_curso'].isin(assoc_filtrada['id_curso'].unique())]

        return cursos_trilha.reset_index(drop=True)

    except Exception as e:
        print(f"Erro ao listar cursos da trilha {id_trilha}: {e}")
        return pd.DataFrame()
    
def listar_trilhas_de_um_curso(id_key_google_sheets, cliente, id_curso):
    """
    Retorna um DataFrame com todas as trilhas associadas a um curso.

    Parâmetros:
    -----------
    id_key_google_sheets : str
        ID da planilha Google Sheets.

    cliente : gspread.Client
        Cliente gspread autenticado.

    id_curso : int
        ID do curso para filtrar as trilhas.

    Retorno:
    --------
    pd.DataFrame
        DataFrame com as trilhas associadas ao curso.
    """
    planilha = cliente.open_by_key(id_key_google_sheets)

    try:
        # Lê associação trilhas -> cursos
        aba_assoc = planilha.worksheet('trilhas_tem_cursos')
        df_assoc = get_as_dataframe(aba_assoc).dropna(how='all')

        # Garante que as colunas numéricas estejam corretas
        df_assoc['id_trilha'] = pd.to_numeric(df_assoc['id_trilha'], errors='coerce').fillna(-1).astype(int)
        df_assoc['id_curso'] = pd.to_numeric(df_assoc['id_curso'], errors='coerce').fillna(-1).astype(int)

        # Filtra associações para o id_curso
        assoc_filtrada = df_assoc[df_assoc['id_curso'] == id_curso]

        if assoc_filtrada.empty:
            print(f"Nenhuma trilha encontrada para o curso id {id_curso}")
            return pd.DataFrame()

        # Lê dados das trilhas
        aba_trilhas = planilha.worksheet('trilhas')
        df_trilhas = get_as_dataframe(aba_trilhas).dropna(how='all')
        df_trilhas['id_trilha'] = pd.to_numeric(df_trilhas['id_trilha'], errors='coerce').fillna(-1).astype(int)

        # Filtra as trilhas associadas ao curso
        trilhas_curso = df_trilhas[df_trilhas['id_trilha'].isin(assoc_filtrada['id_trilha'].unique())]

        return trilhas_curso.reset_index(drop=True)

    except Exception as e:
        print(f"Erro ao listar trilhas do curso {id_curso}: {e}")
        return pd.DataFrame()

# Exemplo de como chamar as funções

In [ ]:
key_google = ID_DA_PLANILHA_GOOGLE_SHEETS
cliente = abrir_cliente_JSON()
id_do_membro=1

print(listar_trilhas_feitas_por_um_membro(key_google, cliente, id_do_membro))

   id_trilha                          nome_trilha  \
0          2            Capacitação Avançada 2025   
1          1  Associate Data Scientist  in Python   

                                                 url  tipo_trilha  \
0  https://app.datacamp.com/learn/career-tracks/c...          1.0   
1  https://app.datacamp.com/learn/career-tracks/a...          0.0   

           data_inicio             data_fim  finalizado  
0  2025-06-01 00:00:00  2025-06-15 00:00:00         1.0  
1  2025-06-01 00:00:00  2025-06-15 00:00:00         1.0  


# Atualizar cursos e trilhas do datacamp
Inicalmente atualiza as sheets de [Trilhas] e de [Cursos], depois disso faz a associação entre os ids de trilhas e cursos, gerando a sheet [Trilhas_tem_Cursos]

In [ ]:
# Autenticação usando o JSON baixado
credenciais = ServiceAccountCredentials.from_json_keyfile_name(LOCAL_CREDENCIAIS_JSON_DE_ACESSO, ESCOPOS) #Credenciais do JSON

cliente = gspread.authorize(credenciais)

# Abre a planilha
planilha = cliente.open_by_key(ID_DA_PLANILHA_GOOGLE_SHEETS)

# Lê todas as abas em um dicionário: {nome_aba: dataframe}
planilhas = {
    aba.title: get_as_dataframe(aba).dropna(how="all")
    for aba in planilha.worksheets() #Esse comando retorna uma lista de objetos aba, ou seja, todas as guias/abas da planilha do Google Sheets.
}

#Acessa as abas especificas Trilhas e Cursos
df_sheet_trilhas_maedev = planilhas['trilhas']
df_sheet_cursos_maedev = planilhas['cursos']

#Atualiza baseado em arquivo local
for trilha in LISTA_DE_TRILHAS_PARA_LER:
    trilhas_atualizadas = pd.read_excel(f'./{PASTA_TRILHAS}/{trilha}') #Essa lista de trilhas para ler é arquivo local (vem do scrapping do datacamp)
    df_sheet_trilhas_maedev = atualizar_trilhas_com_ids(trilhas_atualizadas, df_sheet_trilhas_maedev) #Atualiza o DF de trilhas
    df_sheet_cursos_maedev = atualizar_cursos_com_ids(trilhas_atualizadas, df_sheet_cursos_maedev) #Atualiza o DF de cursos



# Sobrescrevendo o arquivo original (unindo de volta ao DF com todas as abas)
planilhas['trilhas'] = df_sheet_trilhas_maedev
planilhas['cursos'] = df_sheet_cursos_maedev



# Salva todas as abas de volta (sobrescrevendo o arquivo original)
for nome_sheet, df in planilhas.items(): #Para cada aba
    try:
        # Tenta abrir a aba existente
        aba = planilha.worksheet(nome_sheet)
        # Limpa o conteúdo antigo
        aba.clear()
    except gspread.WorksheetNotFound:
        # Se a aba não existir, cria nova
        aba = planilha.add_worksheet(title=nome_sheet, rows=str(len(df)+10), cols=str(len(df.columns)+10))

    # Escreve o DataFrame atualizado na aba
    set_with_dataframe(aba, df)


# Associa as trilhas com os cursos
gerar_trilhas_tem_cursos(cliente, ID_DA_PLANILHA_GOOGLE_SHEETS, PASTA_TRILHAS)


# Criando cookies 
Antes de puxar as trilhas do datacamp, é necessário logar e gerar os cookies

In [ ]:
from playwright.async_api import async_playwright # Rode 'playwright install' no terminal para instalar os navegadores quando for usar pela primeira vez
import pickle
import asyncio
import os
import nest_asyncio # Se estiver rodando no Jupyter ou Colab, use 'nest_asyncio' para permitir chamadas assíncronas

nest_asyncio.apply()  # permite rodar async no Jupyter/Colab

url = "https://app.datacamp.com"
cookies_path = "./cookies/datacamp_cookies.pkl"

async def save_cookies_manually():
    os.makedirs(os.path.dirname(cookies_path), exist_ok=True)

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        await page.goto(url, timeout=60000)
        await context.clear_cookies()

        input("⚠️ Faça login no navegador que abriu. Depois pressione Enter aqui...")

        cookies = await context.cookies()
        with open(cookies_path, "wb") as f:
            pickle.dump(cookies, f)

        print(f"✅ Cookies salvos com sucesso em: {cookies_path}")

# Em Jupyter ou Colab:
await save_cookies_manually()

✅ Cookies salvos com sucesso em: ./cookies/datacamp_cookies.pkl


# Gerando arquivo com os cursos do datacamp
### Gera um xlsx com a ata de cursos de cada trilha dentro da lista LISTA_DE_TRILHAS_PARA_ATUALIZAR

In [ ]:


'''
Mude as urls para atualizar trilhas diferentes do datacamp, 
Para ser mais rápido, atualize uma por vez e eviter atualizar coisas desnecessárias
Funciona tanto para skill tracks quanto carrer tracks
'''
LISTA_DE_TRILHAS_PARA_ATUALIZAR = [
    #trilhas da capacitação básica 2025
    #"https://app.datacamp.com/learn/career-tracks/data-scientist-with-python",
    #'https://app.datacamp.com/learn/skill-tracks/sql-fundamentals',
    #'https://app.datacamp.com/learn/skill-tracks/git-fundamentals',
    #'https://app.datacamp.com/learn/skill-tracks/excel-fundamentals',

    #trilhas da capacitação avançada 2025
    #'https://app.datacamp.com/learn/skill-tracks/time-series-with-python', #curso de Séries temporais
    #'https://app.datacamp.com/learn/skill-tracks/applied-finance-in-python', #curso de Introdução ao Gerenciamento de Risco de Portfólio em Python e curso de Garch
    #'https://app.datacamp.com/learn/skill-tracks/finance-fundamentals-in-python', #curso de Introdução à Análise de Portfólio em python
    'https://app.datacamp.com/learn/career-tracks/machine-learning-scientist-with-python' #contem boa parte dos cursos de machine learning usados
     
]
'''
CURSOS SEM TRILHAS (PRECISAM SER ADICIONADOS MANUALMENTE)
https://app.datacamp.com/learn/courses/introduction-to-optimization-in-python
https://app.datacamp.com/learn/courses/introduction-to-deep-learning-in-python
'''

nest_asyncio.apply()

cookies_path = "./cookies/datacamp_cookies.pkl"
semaphore = asyncio.Semaphore(3)

# Cria o contexto autenticado com os cookies
async def create_authenticated_context(playwright):
    if not os.path.exists(cookies_path):
        print("❌ Arquivo de cookies não encontrado. Faça o login e salve os cookies primeiro.")
        return None, None

    browser = await playwright.chromium.launch(headless=False)
    context = await browser.new_context()

    with open(cookies_path, "rb") as f:
        cookies = pickle.load(f)
    await context.add_cookies(cookies)

    return browser, context

# Extrai os cursos e durações de uma trilha
async def extract_courses(context, url):
    async with semaphore:
        page = await context.new_page()
        try:
            await page.goto(url, timeout=60000)
            await asyncio.sleep(random.uniform(2, 4))

            if "just a moment" not in (await page.content()).lower():
                soup = BeautifulSoup(await page.content(), "html.parser")

                track_element = soup.find("h1")
                track = track_element.get_text(strip=True) if track_element else "N/A"

                projects = {
                    optional.find_parent("div")
                    for optional in soup.select(".mfe-app-learn-hub-1moscjt")
                }

                courses = list([
                    course.get_text(strip=True)
                    for course in soup.select("h3.mfe-app-learn-hub-1yqo1j7")
                    if course.find_parent("div") not in projects
                ])
                await page.close()

                duration = {}
                for course in courses:
                    course_page = course.replace(" ", "-").lower()
                    tmp_url = f"https://app.datacamp.com/learn/courses/{course_page}"
                    page = await context.new_page()
                    try:
                        await page.goto(tmp_url, timeout=60000)
                        await asyncio.sleep(random.uniform(2, 3))

                        soup = BeautifulSoup(await page.content(), "html.parser")
                        duration_element = soup.select_one(".mfe-app-learn-hub-hdd90k")
                        duration[course] = duration_element.get_text(strip=True) if duration_element else "N/A"
                    except Exception as e:
                        print(f"⚠️ Erro ao acessar curso '{course}': {e}")
                        duration[course] = "N/A"
                    finally:
                        await page.close()

                data = [{"Trilha": track, "Curso": course, "Duração": duration[course]} for course in courses]
                print(f"✅ Cursos extraídos com sucesso da trilha: {track}")
                return data
            else:
                print("❌ Cookies expirados ou inválidos. Refaça o login.")
                return None
        except Exception as e:
            print(f"❌ Erro ao extrair curso da página {url}: {e}")
            return None

# Função principal
async def main(url_da_trilha):
    async with async_playwright() as p:
        browser, context = await create_authenticated_context(p)
        if context is None:
            return

        data = await extract_courses(context, url_da_trilha)

        await browser.close()

        if data:
            df = pd.DataFrame(data)
            df['ordem_para_assistir'] = range(len(df)) #ordem de acordo com o html da página
            df['tipo_trilha'] = 0 #0->Trilha do datacamp, 1->trilha personalizada
            df['data_final_para_assistir'] = '' #cursos do datacamp não tem datas especificas, isso só se aplica para trilhas personalizadas
            df['obrigatoriedade_curso'] = 1 #cursos do datacamp por default sao obrigatorios, opcionalidade só se aplica para trilhas personalizadas
            print(df)
            return df
        else:
            print("Nenhum dado extraído.")
            return None


# Loop principal para pegar todas as trilhas da lista
async def rodar_todas_as_trilhas():
    for trilha_do_datacamp in LISTA_DE_TRILHAS_PARA_ATUALIZAR:
        df = await main(trilha_do_datacamp)
        if df is not None:
            df.to_excel((f'trilhas_atualizadas/{df.iloc[0]["Trilha"]}.xlsx'), index=False) #Salva o arquivo com titulo da trilha

#Executa a função
await rodar_todas_as_trilhas()

✅ Cursos extraídos com sucesso da trilha: Machine Learning Scientist in Python
                                  Trilha  \
0   Machine Learning Scientist in Python   
1   Machine Learning Scientist in Python   
2   Machine Learning Scientist in Python   
3   Machine Learning Scientist in Python   
4   Machine Learning Scientist in Python   
5   Machine Learning Scientist in Python   
6   Machine Learning Scientist in Python   
7   Machine Learning Scientist in Python   
8   Machine Learning Scientist in Python   
9   Machine Learning Scientist in Python   
10  Machine Learning Scientist in Python   
11  Machine Learning Scientist in Python   
12  Machine Learning Scientist in Python   
13  Machine Learning Scientist in Python   
14  Machine Learning Scientist in Python   
15  Machine Learning Scientist in Python   
16  Machine Learning Scientist in Python   
17  Machine Learning Scientist in Python   
18  Machine Learning Scientist in Python   
19  Machine Learning Scientist in Python 